## Avaliação do Conditional Random Fields

In [1]:
import operator
import os
import random
import functools
import collections
import joblib
import random
import nltk
import sklearn
import sklearn_crfsuite
import numpy as np
from itertools import chain
from nltk.corpus import PlaintextCorpusReader 
from nltk import sent_tokenize, word_tokenize, pos_tag 
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import KFold

random.seed(1999)

In [2]:
#função do tamanho da palavra (returna True se for maior que 4)
def length(word):
    if len(word) >= 4: 
        tamanho = True
    else:
        tamanho = False
    return tamanho

teste_tagger = joblib.load('POS_tagger_brill.pkl')

def postag(word):
    phrase = word
    postag = teste_tagger.tag(word_tokenize(phrase))
    return postag[0][1]

#tamanho da setenca
def tamsent(sent,i):
    conta = []
    valor = []
    for i in range(len(sent)):
        conta.append(sent[i].count(sent[i][0]))
    valor = sum(conta)
    return valor

#frequencia da palavra na sentenca
def freqwordsent(sent,word):
    conta = []
    valor = []
    for j in range(len(sent)):
        conta.append(sent[j].count(word))
    valor = sum(conta)
    return valor


def word2features(sent, i):
    word = sent[i][0]  
    features = {
        'bias': 1.0,
        'word': word,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[-1:]': word[-1:],        
        'word[:1]': word[:1],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag(word),
        'postag[:2]': postag(word)[:1],
        'postag[:2]': postag(word)[:2],
        'tamanho': length(word),
        'word.isalnum()' : word.isalnum(),
        'len(word)': len(word),
        'tamanho(sent)': tamsent(sent,i),
        'freqwordsent' : freqwordsent(sent,word),   
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word': word1,
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag(word1),
            '-1:postag[:2]': postag(word1)[:1],
            '-1:postag[:2]': postag(word1)[:2],
            '-1:word[-3:]': word1[-3:],
            '-1:word[-2:]': word1[-2:],
            '-1:word[-1:]': word1[-1:],
            '-1:word[:1]': word1[:1],
            '-1:word[:2]': word1[:2],
            '-1:word[:3]': word1[:3],
            '-1:len(word)': len(word1),
            '-1:word.isalnum()' : word1.isalnum(),
        })
    else:
        features['Inicio'] = True

    if i > 1:
        word2 = sent[i-2][0]
        features.update({
            '-2:word': word2,
            '-2:word.lower()': word2.lower(),
            '-2:word.istitle()': word2.istitle(),
            '-2:word.isdigit()': word2.isdigit(),
            '-2:word.isupper()': word2.isupper(),
            '-2:postag': postag(word2),
            '-2:postag[:2]': postag(word2)[:2],
            '-2:word[-3:]': word2[-3:],
            '-2:word[-2:]': word2[-2:],
            '-2:word[-1:]': word2[-1:],
            '-2:word[:1]': word2[:1],
            '-2:word[:2]': word2[:2],
            '-2:word[:3]': word2[:3],
            '-2:len(word)': len(word2),
            '-2:word.isalnum()' : word2.isalnum(),

        })
    if i < len(sent)-1:
        word3 = sent[i+1][0]
        features.update({
            '+1:word': word3,
            '+1:word.lower()': word3.lower(),
            '+1:word.istitle()': word3.istitle(),
            '+1:word.isdigit()': word3.isdigit(),
            '+1:word.isupper()': word3.isupper(),
            '+1:postag': postag(word3),
            '+1:postag[:2]': postag(word3)[:2],
            '+1:word[-3:]': word3[-3:],
            '+1:word[-2:]': word3[-2:],
            '+1:word[-1:]': word3[-1:],
            '+1:word[:1]': word3[:1],
            '+1:word[:2]': word3[:2],
            '+1:word[:3]': word3[:3],
            '+1:len(word)': len(word3),
            '+1:word.isalnum()' : word3.isalnum()
            })
    else:
        features['Final'] = True
   
    if i < len(sent)-2:
        word4 = sent[i+2][0]
        features.update({
            '+2:word': word4,
            '+2:word.lower()': word4.lower(),
            '+2:word.istitle()': word4.istitle(),
            '+2:word.isdigit()': word4.isdigit(),
            '+2:word.isupper()': word4.isupper(),
            '+2:postag': postag(word4),
            '+2:postag[:2]': postag(word4)[:2],
            '+2:word[-3:]': word4[-3:],
            '+2:word[-2:]': word4[-2:],
            '+2:word[-1:]': word4[-1:],
            '+2:word[:1]': word4[:1],
            '+2:word[:2]': word4[:2],
            '+2:word[:3]': word4[:3],
            '+2:len(word)': len(word4),
            '+2:word.isalnum()' : word4.isalnum()
        })     

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

lista = [
        'bias',
        'word',
        'word.lower()',
        'word[-3:]',
        'word[-2:]',
        'word[-1:]',
        'word[:1]',
        'word[:2]',
        'word[:3]',
        'word.isupper()',
        'word.istitle()',
        'word.isdigit()',
        'postag','postag[:2]',
        'postag[:2]',
        'tamanho',
        'word.isalnum()',
        'len(word)',
        'tamanho(sent)',
        'freqwordsent',
        '-1:word',
        '-1:word.lower()',
        '-1:word.istitle()',
        '-1:word.isdigit()',
        '-1:word.isupper()',
        '-1:postag',
        '-1:postag[:2]',
        '-1:postag[:2]',
        '-1:word[-3:]',
        '-1:word[-2:]',
        '-1:word[-1:]',
        '-1:word[:1]',
        '-1:word[:2]',
        '-1:word[:3]',
        '-1:len(word)',
        '-1:word.isalnum()',
        '-2:word',
        '-2:word.lower()',
        '-2:word.istitle()',
        '-2:word.isdigit()',
        '-2:word.isupper()',
        '-2:postag',
        '-2:postag[:2]',
        '-2:word[-3:]',
        '-2:word[-2:]',
        '-2:word[-1:]',
        '-2:word[:1]',
        '-2:word[:2]',
        '-2:word[:3]',
        '-2:len(word)',
        '-2:word.isalnum()',
        '+1:word',
        '+1:word.lower()',
        '+1:word.istitle()',
        '+1:word.isdigit()',
        '+1:word.isupper()',
        '+1:postag',
        '+1:postag[:2]',
        '+1:word[-3:]',
        '+1:word[-2:]',
        '+1:word[-1:]',
        '+1:word[:1]',
        '+1:word[:2]',
        '+1:word[:3]',
        '+1:len(word)',
        '+1:word.isalnum()',
        '+2:word',
        '+2:word.lower()',
        '+2:word.istitle()',
        '+2:word.isdigit()',
        '+2:word.isupper()',
        '+2:postag',
        '+2:postag[:2]',
        '+2:word[-3:]',
        '+2:word[-2:]',
        '+2:word[-1:]',
        '+2:word[:1]',
        '+2:word[:2]',
        '+2:word[:3]',
        '+2:len(word)',
        '+2:word.isalnum()'
]

In [3]:
f"Número de atributos do modelo: {len(lista)}"

'Número de atributos do modelo: 81'

In [4]:
DIR_PL = "./dados-categorias/PLs/"  
DIR_ST = "./dados-categorias/STs/"
DIR_C = "./dados-categorias/Comentarios/"
# DIR_PL = "./dados-tipos/PLs/"  
# DIR_ST = "./dados-tipos/STs/"
# DIR_C = "./dados-tipos/Comentarios/"

In [5]:
all_files_pl = [DIR_PL+f for f in os.listdir(DIR_PL)]
all_files_st = [DIR_ST+f for f in os.listdir(DIR_ST)]
all_files_c = [DIR_C+f for f in os.listdir(DIR_C)]

In [6]:
def process_conll_file(location:str)->list:
    with open(location, "r") as f:
        data = f.read()
    data = data.split("\n\n")
    data = list(map(lambda x:x.split("\n"), data))
    data.pop()
    data = list(map(lambda x:[operator.itemgetter(*[0, -1])(y.split(" ")) for y in x], data))
    return data

def combine_files(locations:list)->list:
    extended = []
    for f in locations:
        extended.extend(process_conll_file(f))
    return extended

In [7]:
# Divisão entre Conjuntos de Treinamento e de Teste
# PL's
train_size_pl = int(0.75*len(all_files_pl))
random.shuffle(all_files_pl)
train_files_pl = all_files_pl[:train_size_pl]
test_files_pl = all_files_pl[train_size_pl:]
    
train_pl = combine_files(train_files_pl)
test_pl = combine_files(test_files_pl)

# ST's
all_data_st = combine_files(all_files_st)
random.shuffle(all_data_st)
train_size_st = int(0.75*len(all_data_st))
train_st = all_data_st[:train_size_st]
test_st = all_data_st[train_size_st:]

# Comentarios
all_data_c = combine_files(all_files_c)
random.shuffle(all_data_c)
train_size_c = int(0.75*len(all_data_c))
train_c = all_data_c[:train_size_c]
test_c = all_data_c[train_size_c:]

print(f"Número de Sentenças no Conjunto de Treinamento (PL): {len(train_pl)}")
print(f"Número de Sentenças no Conjunto de Teste (PL): {len(test_pl)}")
print(f"Número de Sentenças no Conjunto de Treinamento (ST): {len(train_st)}")
print(f"Número de Sentenças no Conjunto de Teste (ST): {len(test_st)}")
print(f"Número de Sentenças no Conjunto de Treinamento (Comentarios): {len(train_c)}")
print(f"Número de Sentenças no Conjunto de Teste (Comentarios): {len(test_c)}")

Número de Sentenças no Conjunto de Treinamento (PL): 6828
Número de Sentenças no Conjunto de Teste (PL): 2698
Número de Sentenças no Conjunto de Treinamento (ST): 592
Número de Sentenças no Conjunto de Teste (ST): 198
Número de Sentenças no Conjunto de Treinamento (Comentarios): 725
Número de Sentenças no Conjunto de Teste (Comentarios): 242


In [8]:
## Validação cruzada com k=5
# to_store = "CRF-CV"
    
# kfold = KFold(n_splits=5)
# train = np.array(train_c, dtype=object)
# i = 1
# for t, tt in kfold.split(train):
#     to_train = train[t].tolist()
#     to_val = train[tt].tolist()
    
#     X_train = [sent2features(s) for s in to_train]
#     y_train = [sent2labels(s) for s in to_train]

#     X_test = [sent2features(s) for s in to_val]
    
#     crf = sklearn_crfsuite.CRF(
#         algorithm='lbfgs',
#         c1=0.9,
#         c2=0.4,
#         max_iterations=100,
#         all_possible_transitions=True
#     )
#     crf.fit(X_train, y_train)
    
#     ycrf = crf.predict(X_test)
    
#     crf_file = ""
#     for preds, true in zip(ycrf, to_val):
#         for j in range(len(preds)):
#             crf_file += true[j][0] + " " + true[j][1] + " " + preds[j] + "\n"
#         crf_file += "\n"
#     with open(f"./{to_store}/predictions_file_{i}", "w") as f:
#         f.write(crf_file)
#     print(i)
#     i += 1
    

# Experimento II: Transferência de Conhecimento entre diferentes corpus

# Extração Features PL

In [31]:
%%time
X_train_pl = [sent2features(s) for s in train_pl]
y_train_pl = [sent2labels(s) for s in train_pl]

X_test_pl = [sent2features(s) for s in test_pl]
y_test_pl = [sent2labels(s) for s in test_pl]

CPU times: user 4min 33s, sys: 604 ms, total: 4min 34s
Wall time: 4min 34s


# Extração Features ST

In [32]:
%%time
X_train_st = [sent2features(s) for s in train_st]
y_train_st = [sent2labels(s) for s in train_st]

X_test_st = [sent2features(s) for s in test_st]
y_test_st = [sent2labels(s) for s in test_st]

CPU times: user 2min 53s, sys: 544 ms, total: 2min 53s
Wall time: 2min 53s


# Extração Features Comentários

In [9]:
%%time
X_train_c = [sent2features(s) for s in train_c]
y_train_c = [sent2labels(s) for s in train_c]

X_test_c = [sent2features(s) for s in test_c]
y_test_c = [sent2labels(s) for s in test_c]

CPU times: user 3min 18s, sys: 554 ms, total: 3min 18s
Wall time: 3min 18s


# Treinamento

In [ ]:
%%time
# Treinamento PL
crf_pl = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=500,
    all_possible_transitions=True
)
crf_pl.fit(X_train_pl, y_train_pl)

In [ ]:
%%time
# Treinamento ST
crf_st = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=500,
    all_possible_transitions=True
)
crf_st.fit(X_train_st, y_train_st)

In [28]:
%%time
# Treinamento Comentários
crf_c = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.0,
    c2=0.0,
    max_iterations=500,
    all_possible_transitions=True
)
crf_c.fit(X_train_c, y_train_c)

CPU times: user 3min 40s, sys: 60.2 ms, total: 3min 40s
Wall time: 3min 40s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.0, c2=0.0,
    keep_tempfiles=None, max_iterations=500)

In [33]:
# CRF treinado nos Comentários
y_crf_c_pl = crf_c.predict(X_test_pl)
y_crf_c_st = crf_c.predict(X_test_st)
y_crf_c_c = crf_c.predict(X_test_c)

In [34]:
store_results(test_pl, y_crf_c_pl, "CRF-C-PL")
store_results(test_st, y_crf_c_st, "CRF-C-ST")
store_results(test_c, y_crf_c_c, "CRF-C-C")

In [ ]:
%%time
# Treinamento PL + ST
X_train_plst = X_train_pl + X_train_st
y_train_plst = y_train_pl + y_train_st

crf_plst = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=500,
    all_possible_transitions=True
)
crf_plst.fit(X_train_plst, y_train_plst)

In [ ]:
%%time
# Treinamento PL + Comentários
X_train_plc = X_train_pl + X_train_c
y_train_plc = y_train_pl + y_train_c

crf_plc = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=500,
    all_possible_transitions=True
)
crf_plc.fit(X_train_plc, y_train_plc)

In [ ]:
%%time
# Treinamento ST + Comentários
X_train_stc = X_train_st + X_train_c
y_train_stc = y_train_st + y_train_c

crf_stc = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=500,
    all_possible_transitions=True
)
crf_stc.fit(X_train_stc, y_train_stc)

In [ ]:
%%time
# Treinamento PL + ST + Comentários
X_train_all = X_train_pl + X_train_st + X_train_c
y_train_all = y_train_pl + y_train_st + y_train_c

crf_all = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.9,
    c2=0.4,
    max_iterations=500,
    all_possible_transitions=True
)
crf_all.fit(X_train_all, y_train_all)

# Teste

In [ ]:
# CRF treinado nos PL's
y_crf_pl_pl = crf_pl.predict(X_test_pl)
y_crf_pl_st = crf_pl.predict(X_test_st)
y_crf_pl_c = crf_pl.predict(X_test_c)

# CRF treinado nas ST's
y_crf_st_pl = crf_st.predict(X_test_pl)
y_crf_st_st = crf_st.predict(X_test_st)
y_crf_st_c = crf_st.predict(X_test_c)

# CRF treinado nos Comentários
y_crf_c_pl = crf_c.predict(X_test_pl)
y_crf_c_st = crf_c.predict(X_test_st)
y_crf_c_c = crf_c.predict(X_test_c)

# CRF treinado nos PL's + ST's
y_crf_plst_pl = crf_plst.predict(X_test_pl)
y_crf_plst_st = crf_plst.predict(X_test_st)
y_crf_plst_c = crf_plst.predict(X_test_c)

# CRF treinado nos PL's + Comentários
y_crf_plc_pl = crf_plc.predict(X_test_pl)
y_crf_plc_st = crf_plc.predict(X_test_st)
y_crf_plc_c = crf_plc.predict(X_test_c)

# CRF treinado nos ST's + Comentários
y_crf_stc_pl = crf_stc.predict(X_test_pl)
y_crf_stc_st = crf_stc.predict(X_test_st)
y_crf_stc_c = crf_stc.predict(X_test_c)

In [ ]:
y_crf_all_pl = crf_all.predict(X_test_pl)
y_crf_all_st = crf_all.predict(X_test_st)
y_crf_all_c = crf_all.predict(X_test_c)

In [13]:
def store_results(test, y_pred, filename):
    content = ""
    for preds, true in zip(y_pred, test):
        for j in range(len(preds)):
            content += true[j][0] + " " + true[j][1] + " " + preds[j] + "\n"
        content += "\n"
    with open(filename, "w") as f:
        f.write(content)

In [ ]:
store_results(test_pl, y_crf_pl_pl, "CRF-PL-PL")
store_results(test_pl, y_crf_st_pl, "CRF-ST-PL")
store_results(test_pl, y_crf_c_pl, "CRF-C-PL")
store_results(test_pl, y_crf_plst_pl, "CRF-PLST-PL")
store_results(test_pl, y_crf_plc_pl, "CRF-PLC-PL")
store_results(test_pl, y_crf_stc_pl, "CRF-STC-PL")

store_results(test_st, y_crf_pl_st, "CRF-PL-ST")
store_results(test_st, y_crf_st_st, "CRF-ST-ST")
store_results(test_st, y_crf_c_st, "CRF-C-ST")
store_results(test_st, y_crf_plst_st, "CRF-PLST-ST")
store_results(test_st, y_crf_plc_st, "CRF-PLC-ST")
store_results(test_st, y_crf_stc_st, "CRF-STC-ST")

store_results(test_c, y_crf_pl_c, "CRF-PL-C")
store_results(test_c, y_crf_st_c, "CRF-ST-C")
store_results(test_c, y_crf_c_c, "CRF-C-C")
store_results(test_c, y_crf_plst_c, "CRF-PLST-C")
store_results(test_c, y_crf_plc_c, "CRF-PLC-C")
store_results(test_c, y_crf_stc_c, "CRF-STC-C")

In [ ]:
store_results(test_pl, y_crf_all_pl, "CRF-ALL-PL")
store_results(test_st, y_crf_all_st, "CRF-ALL-ST")
store_results(test_c, y_crf_all_c, "CRF-ALL-C")